In [1]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.utils.data
import torchvision
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
from sklearn.preprocessing import MinMaxScaler
import time

In [2]:
ti = time.time()

In [3]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Modified by M. Romero.
    Original: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [4]:
mpl.rc("font", size=20)

In [77]:
cuda = True
device = torch.device("cuda:0" if cuda and torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#### Carga del dataset

#### Digits

In [5]:
digits = load_digits()
targets = digits.target
data = digits.data
classes = digits.target_names

#### Division train-val-test

In [6]:
# train-validation-test split
nsamples, ndim = data.shape
nlabels = len(classes)
indexes = np.arange(nsamples)
train_idx, test_idx = train_test_split(indexes, test_size=0.2)
train_idx, val_idx = train_test_split(train_idx, test_size=0.1/0.8)

print("train {:.0f} %".format(len(train_idx)/nsamples*100))
print("val {:.0f} %".format(len(val_idx)/nsamples*100))
print("test {:.0f} %".format(len(test_idx)/nsamples*100))

x_train = data[train_idx]
x_val = data[val_idx]
x_test = data[test_idx]

y_train = targets[train_idx]
y_val = targets[val_idx]
y_test = targets[test_idx]

train 70 %
val 10 %
test 20 %


#### Normalizacion de datos

In [7]:
#scaler = StandardScaler() # (x - mu)/std, (mu, std) = (0, 1)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

#### Declaracion de tensores y dispositivo de trabajo

In [6]:
cuda = False
device = torch.device("cuda:0" if cuda and torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
x_train = torch.tensor(x_train, device=device, dtype=torch.float)
x_val = torch.tensor(x_val, device=device, dtype=torch.float)
x_test = torch.tensor(x_test, device=device, dtype=torch.float)

y_train = torch.tensor(y_train, device=device, dtype=torch.long) # ojo que al utilizar CrossEntropyLoss 
y_val = torch.tensor(y_val, device=device, dtype=torch.long)     # la entrada debe ser tipo long
y_test = torch.tensor(y_test, device=device, dtype=torch.long)

#### Declaracion de dataset

In [10]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

#### Construccion de red neuronal

In [22]:
class MLP(torch.nn.Module):
    def __init__(self, ninput, nhidden, nout, bn=False, do=False):
        super(MLP, self).__init__()
        layers = []
        layers.append(torch.nn.Linear(ninput, nhidden))
        layers.append(torch.nn.BatchNorm1d(nhidden)) if bn else 0
        layers.append(torch.nn.Dropout(0.5)) if do else 0
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Linear(nhidden, nout))
        self.mlp = torch.nn.Sequential(*layers)


    def forward(self, x):
        return self.mlp(x)
    
    
class AE(torch.nn.Module):
    def __init__(self, nin, nl, bn=False, do=False):
        super(AE, self).__init__()
        self.enc1 = torch.nn.Linear(nin, 32)
        self.enc2 = torch.nn.Linear(32, nl)
        self.dec1 = torch.nn.Linear(32, nin)
        self.dec2 = torch.nn.Linear(nl, 32)
        self.relu = torch.nn.ReLU()
        self.sig = torch.nn.Sigmoid()
    
    def encode(self, x):
        e1 = self.relu(self.enc1(x))
        return self.enc2(e1)
    
    def decode(self, u):
        d1 = self.relu(self.dec2(u))
        return self.sig(self.dec1(d1))
        
    def forward(self, x):
        l = self.encode(x)
        r = self.decode(l)
        return l, r

#### Decalaracion de hiper parametros

In [88]:
lr = 2e-4
wd = 0.
bs = 100
neurons = 100
epochs = 10

#### MNIST

In [115]:
transform = [torchvision.transforms.ToTensor()]

N = 10000

trainset = torchvision.datasets.MNIST(root="../datasets/mnist/train", train=True, download=True, transform=torchvision.transforms.Compose(transform))
testset = torchvision.datasets.MNIST(root="../datasets/mnist/test", train=False, download=True, transform=torchvision.transforms.Compose(transform))

train_tensor = torch.tensor(trainset.data)[:N].float().to(device).reshape(N, -1)/255.
test_tensor = torch.tensor(testset.data)[:N].float().to(device).reshape(N, -1)/255.

trainDataset = torch.utils.data.TensorDataset(train_tensor, trainset.targets[:N].to(device))
testDataset = torch.utils.data.TensorDataset(test_tensor, testset.targets[:N].to(device))

trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=4, shuffle=True)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=4, shuffle=True)

c:\users\mauricio\anaconda3\envs\color\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\mauricio\anaconda3\envs\color\lib\site-packages\ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


#### Declaracion de dataloader

In [117]:
"""
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=True)
"""

'\ntrain_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)\nval_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=True)\ntest_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=True)\n'

#### Funcion de perdida

In [118]:
cel = torch.nn.CrossEntropyLoss().to(device)
bcel = torch.nn.BCELoss().to(device)
msel = torch.nn.MSELoss().to(device)

#### Funcion de entrenamiento

In [119]:
def train_my_model(epochs, model, optimizer, loss_function, trainloader, valloader, testloader):
    losses = np.zeros((epochs, 3))
    best_val_loss = np.inf
    for epoch in range(epochs):
        train_loss = 0
        val_loss = 0
        test_loss = 0
        model.train()
        for i, batch in enumerate(trainloader):
            x_in, y_in = batch
            _, x_out = model(x_in)
            loss = loss_function(x_out, x_in)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= i + 1
        model.eval()
        with torch.no_grad():
            if valloader != None:
                for i, batch in enumerate(valloader):
                    x_in, y_in = batch
                    _, x_out = model(x_in)
                    loss = loss_function(x_out, x_in)
                    val_loss += loss.item()
                val_loss /= i + 1
            for i, batch in enumerate(testloader):
                x_in, y_in = batch
                _, x_out = model(x_in)
                loss = loss_function(x_out, x_in)
                test_loss += loss.item()
            test_loss /= i + 1
        losses[epoch] = [train_loss, val_loss, test_loss]
        print("Epoch {} Train loss {:.3f} Val loss {:.3f} Test loss {:.3f}".format(epoch, train_loss, val_loss, test_loss))
        if valloader != None:
            if val_loss < best_val_loss:
                print("Saving")
                torch.save(model.state_dict(), "models/ae.pth")
                best_val_loss = val_loss
        else:
            if test_loss < best_val_loss:
                print("Saving")
                torch.save(model.state_dict(), "models/ae.pth")
                best_val_loss = test_loss
    return losses


def plot_my_loss(loss, title, ylabel="mean cross entropy"):
    plt.figure(figsize=(20, 5))
    plt.plot(loss[:, 0], color="navy", label="train")
    plt.plot(loss[:, 1], color="green", label="val")
    plt.plot(loss[:, 2], color="red", label="test")
    plt.xlabel("epochs")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.ylim([0, 2])
    return


def eval_my_model(model, test_data):
    model.eval()
    with torch.no_grad():
        xl, x_out = model(test_data)
    return xl.cpu().numpy(), x_out.cpu().numpy()

In [120]:
model_bce = AE(train_tensor.shape[1], 2).to(device)
optimizer_bce = torch.optim.Adamax(model_bce.parameters(), lr=lr, weight_decay=wd)
loss_bce = train_my_model(epochs, model_bce, optimizer_bce, bcel, trainLoader, None, testLoader)
model_bce.load_state_dict(torch.load("models/ae.pth"))

Epoch 0 Train loss 0.334 Val loss 0.000 Test loss 0.275
Saving


KeyboardInterrupt: 

In [ ]:
model_mse = AE(train_tensor.shape[1], 2).to(device)
optimizer_mse = torch.optim.Adamax(model_mse.parameters(), lr=lr, weight_decay=wd)
loss_mse = train_my_model(epochs, model_mse, optimizer_mse, msel, trainLoader, None, testLoader)
model_mse.load_state_dict(torch.load("models/ae.pth"))

In [ ]:
xl_bce, x_out_bce = eval_my_model(model_bce, test_tensor)
xl_mse, x_out_mse = eval_my_model(model_mse, test_tensor)

In [ ]:
plt.gray()
plt.figure()
rs = 28
i1 = np.random.randint(0, test_tensor.shape[0])
i2 = np.random.randint(0, test_tensor.shape[0])
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(4*4, 4))
ax[0].set_title("real")
ax[0].imshow(test_tensor[i1].cpu().numpy().reshape((rs, rs)))
ax[1].set_title("bce_recon")
ax[1].imshow(x_out_bce[i1].reshape((rs, rs)))
ax[2].set_title("real")
ax[2].imshow(test_tensor[i2].cpu().numpy().reshape((rs,rs)))
ax[3].set_title("bce_recon")
ax[3].imshow(x_out_bce[i2].reshape((rs, rs)))
plt.figure()
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(4*4, 4))
ax[0].set_title("real")
ax[0].imshow(test_tensor[i1].cpu().numpy().reshape((rs, rs)))
ax[1].set_title("mse_recon")
ax[1].imshow(x_out_mse[i1].reshape((rs, rs)))
ax[2].set_title("real")
ax[2].imshow(test_tensor[i2].cpu().numpy().reshape((rs, rs)))
ax[3].set_title("mse_recon")
ax[3].imshow(x_out_mse[i2].reshape((rs, rs)))